<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/Neural_networks/NLP/Langchain/gemma2_2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Standard approach

In [7]:
import os
os.environ["HUGGING_FACE_HUB_TOKEN"] = userdata.get('HF_TOKEN')


In [8]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

tokenizer_config.json:   0%|          | 0.00/46.9k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [9]:
messages = [{"role": "user", "content": "Briefly explain the concept of flash attention and its difference from original attention mechanism."}]
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=256)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
Briefly explain the concept of flash attention and its difference from original attention mechanism.<end_of_turn>
* **Flash Attention** is a novel attention mechanism that aims to improve the efficiency of attention computation by reducing the number of computations required. It achieves this by using a "flash" of attention computation, where the attention weights are calculated for a small subset of the input sequence. 

**Original Attention Mechanism** (e.g., self-attention in Transformer) calculates attention weights for all possible pairs of tokens in the sequence. This can be computationally expensive, especially for long sequences.

**Key Differences:**

* **Computational Efficiency:** Flash attention significantly reduces the number of computations compared to original attention mechanisms.
* **Memory Efficiency:** Flash attention requires less memory to store attention weights, as it only needs to store the attention weights for the subset of tokens.
* 

In [10]:
!nvidia-smi

Thu Aug  1 17:23:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0              30W /  70W |   5405MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [12]:

def print_gpu_memory_usage():
    print(f"Allocated: {torch.cuda.memory_allocated() / 1024 ** 3:.2f} GB")
    print(f"Reserved: {torch.cuda.memory_reserved() / 1024 ** 3:.2f} GB")


def inference_gemma(content, model=model, tokenizer=tokenizer):

  # Prepare the input
  messages = [{"role": "user", "content": content}]
  input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")

  # Check GPU memory usage after preparing input
  print("\nGPU memory usage after preparing input:")
  print_gpu_memory_usage()

  # Generate response
  outputs = model.generate(**input_ids, max_new_tokens=256)

  # Check GPU memory usage after generation
  print("\nGPU memory usage after generation:")
  print_gpu_memory_usage()

  # Decode and print the output
  results = tokenizer.decode(outputs[0])
  print(result)

In [13]:
%%time
inference_gemma("What is the main idea behind flash attention?")

Initial GPU memory usage:
Allocated: 4.91 GB
Reserved: 5.15 GB

GPU memory usage after preparing input:
Allocated: 4.91 GB
Reserved: 5.15 GB

GPU memory usage after generation:
Allocated: 4.91 GB
Reserved: 5.15 GB
<bos><start_of_turn>user
What is the main idea behind flash attention?<end_of_turn>
* **Attention is a mechanism that allows a model to focus on specific parts of the input sequence.**
* **Flash attention is a technique that speeds up attention computation by using a "flash" of attention.**
* **Flash attention is a type of attention mechanism that uses a single attention head to process the entire input sequence.**
* **Flash attention is a technique that reduces the number of attention heads used in a model.**

**The correct answer is:**

**Flash attention is a technique that speeds up attention computation by using a "flash" of attention.**

**Explanation:**

Flash attention is a specific approach to attention mechanisms that aims to improve efficiency. It achieves this by f